In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns

# makes sure charts appear in the notebook rather than a popup window
%matplotlib inline

In [78]:
pd.options.display.max_columns = 60
df = pd.read_csv('./carnivorous_diet.csv', encoding='latin1')

/var/folders/_s/5ccjvt852hv4ddn7j5mrvy1m0000gn/T/ipykernel_16053/1694165870.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./carnivorous_diet.csv', encoding='latin1')


In [79]:
# Remove index duplicate (recordID) and columns made of 99% 'Unknown's
df.drop(columns=['recordID', 'lifeStageCarni'], inplace=True)


In [80]:
# Remove all columns whose N/A represent more than half the dataset
def is_majority_na(columnName):
    return df[columnName].isna().sum() > len(df) / 2

for i in df.columns:
    if (is_majority_na(i)):
        print(f'Dropping column {i}')
        df.drop(columns=[i], inplace=True)



Dropping column speciesPrey
Dropping column scientificNamePrey
Dropping column percentageError
Dropping column sampleSizeKillsPreyItems
Dropping column startDayOfYear
Dropping column endDayOfYear
Dropping column minimumElevationInMeters
Dropping column maximumElevationInMeters
Dropping column geographicRegion
Dropping column stateProvince
Dropping column county
Dropping column municipality
Dropping column verbatimLocality
Dropping column protectedAreaHigher
Dropping column protectedAreaLower
Dropping column islandGroup
Dropping column island


In [81]:
# Remove typos in commonNamePrey.
df.drop(index=[*df.loc[df['commonNamePrey'] == '0.1-0.3 kg rodent'].index,
        *df.loc[df['commonNamePrey'] == '0.3-0.5 kg rodent'].index], inplace=True)
